In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
from selenium import webdriver

In [2]:
page='https://www.ynet.co.il/home/0,7340,L-4269,00.html'
respones=requests.get(page)
page=bs(respones.text,'html.parser')



In [3]:
###Scarping Archive URLS
urls_possible={}
for table in page.find_all('table',{'class':'classMainTableBox'}):
    for data in table.find_all('td'):
        if data.a != None:
            urls_possible[data.a.text]={}
            urls_possible[data.a.text]['main']=data.a.get('href')
            



In [4]:
### Getting Only 2020 Articles From archive
### Can Be simply change by looking archive and change the numbers that ownd in archive and wanted to scrap
main_page = 'https://www.ynet.co.il'
for name, url in urls_possible.items():
    respones = requests.get(main_page + url['main'])
    page = bs(respones.text, 'html.parser')
    if page.find('td', {'class': 'classMainTitle'}):
        for url in page.find_all('td', {'class': 'classMainTitle'}):
            if '2020' in url.text:
                if (url.find('a', {'class': 'indexw'})):
                    urls_possible[name]['2020'] = url.a.get('href')
                else:
                    urls_possible[name]['2020'] = None

    

         
                
    
urls_possible


{'חדשות': {'main': '/home/0,7340,L-4269-199-551,00.html',
  '2020': '/home/0,7340,L-4269-199-551-202000-1,00.html'},
 'מבזקים': {'main': '/home/0,7340,L-4269-3283-184,00.html'},
 'פוליטי מדיני': {'main': '/home/0,7340,L-4269-109-185,00.html',
  '2020': '/home/0,7340,L-4269-109-185-202000-1,00.html'},
 'מדיני': {'main': '/home/0,7340,L-4269-890-315,00.html',
  '2020': '/home/0,7340,L-4269-890-315-202000-1,00.html'},
 'המערכת הפוליטית': {'main': '/home/0,7340,L-4269-891-317,00.html',
  '2020': '/home/0,7340,L-4269-891-317-202000-1,00.html'},
 'צבא וביטחון': {'main': '/home/0,7340,L-4269-141-344,00.html',
  '2020': '/home/0,7340,L-4269-141-344-202000-1,00.html'},
 'פלסטינים': {'main': '/home/0,7340,L-4269-3259-4172,00.html',
  '2020': '/home/0,7340,L-4269-3259-4172-202000-1,00.html'},
 'חדשות בארץ': {'main': '/home/0,7340,L-4269-32-187,00.html',
  '2020': '/home/0,7340,L-4269-32-187-202000-1,00.html'},
 'כללי': {'main': '/home/0,7340,L-4269-200-553,00.html',
  '2020': '/home/0,7340,L-4269

In [22]:
### Creating a Soup for scrapping
def creat_new_soup(link):
    respones = requests.get(link)
    page = bs(respones.text, 'html.parser')
    return page

## Checking if theres a New web To scrap
## Found out empty page owns "td" 114 so we have to check it
## need Selenium to active the JavaScript Without it cant find 
## put the web driver path where you execute it from
def check_if_next_page(soup,link):
    if(len(soup.find_all('td'))>114):
        if(soup.find('table',{'cellspacing':'1'})):
            driver=webdriver.Chrome(executable_path=r"C:\Users\Dima\git\repository\My_first_java_gui\chromedriver.exe")
            driver.get(link)
            soup = bs(driver.page_source, 'html.parser')
            link=soup.find('table',{'cellspacing':'1'})
            for path in link.find_all('td',{'align':'left'}):
                if(path.find('a',{'href':True})):
                    return path.a.get('href')
            return False
        else:
            return False
    else:
        return False
###Main scrpping script
    
def scrap_info(soup,subject,link,index,dico=None):
    main_link='https://www.ynet.co.il'
    part_of_link=''
    full_text=''
    values={index:{'Article_Name':None,'author':None,'Time':None,'link':None,'text':None,'subject':subject}}
    if(len(soup.find_all('td'))>114):               # Empty page of zero articals has 114 td's in it if it bigger then its valied 
        table=soup.find('td',{'class':'ghciArticleIndex1'})
        for value in table.find_all('td',{'dir':'rtl'}):
            if(value.find('a',{'class':'smallheader'})):
                values[index]['Article_Name']=value.find('a',{'class':'smallheader'}).text
                values[index]['link']=value.find('a',{'class':'smallheader'}).get('href')
                part_of_link=value.find('a',{'class':'smallheader'}).get('href')
                Article=creat_new_soup(main_link+part_of_link)
                if(Article.find('div',{'class':'authors'})):
                    values[index]['author']=Article.find('div',{'class':'authors'}).text
                else:
                    if(Article.find('div',{'class':'art_header_footer'})):
                        values[index]['author']=Article.find('span',{'style':'padding-left:8px;'}).text
                    values[index]['author']=None
                if(Article.find('span',{'class':'date'})):
                    values[index]['Time']=Article.find('span',{'class':'date'}).span.get('data-wcmdate')
                else:
                    if(Article.find('span',{'style':'padding-right: 5px;'})):
                        values[index]['Time']=Article.find('span',{'style':'padding-right: 5px;'}).text
                    values[index]['Time']=None
                if(Article.find_all('span',{'data-text':'true'})):
                    for text in Article.find_all('span',{'data-text':'true'}):
                        full_text+=text.text
                    values[index]['text']=full_text
                else:
                        for script in Article(["script", "style"]):
                            script.decompose()                                ##2 Styles of sits needs 2 methodes of scraping
                        for text in Article.findAll('p'):
                            full_text+=text.get_text(" ", strip=True)
                        values[index]['text']=full_text
                index+=1
                full_text=''
                values.update({index:{'Article_Name':None,'author':None,'Time':None,'link':None,'text':None,'subject':subject}})
        check=check_if_next_page(soup,link)# Checking if have Valied URL if yes using recursion to scrap the next page
        print(values)
        if(check!=False):
            new_soup=creat_new_soup(main_link+check) 
            scrap_info(new_soup,subject,main_link+check,index,values)# recursion Scrapping the site if next page is aviable
        return values
        ##this else reffers that the page is empty so return the data the collected                 
    else:
        return values
    



scrapped_info={}
for subject,year in urls_possible.items():
    for DateOfurl,url in year.items():
        ## DateOfurl owns where url taken from and the target date 
        ## main is where url is taken from
        if( (DateOfurl!='main') & (url!= None) ):
            print("ScrapingSubject:%s \nUrl:%s \nYear:%d"%(subject,url,int(DateOfurl)))
            soup=creat_new_soup(main_page+url)
            new_info=scrap_info(soup,subject,main_page+url,1)
            scrapped_info[subject]=new_info

ScrapingSubject:חדשות 
Url:/home/0,7340,L-4269-199-551-202000-1,00.html 
Year:2020


C:\Users\Dima\anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


{1: {'Article_Name': 'מאזדה 3 סדאן תשווק מעתה גם בגרסה מוזלת', 'author': 'רן סגל|', 'Time': '2020-01-01T15:09:29.752Z', 'link': '/wheels/news/article/SJbuAQ5y8', 'text': 'ארבעה חודשים לאחר שמאזדה 3 החדשה (דור 4) הגיעה לישראל כשהיא מצוידת בתגי-מחיר גבוהים יחסית שהחלו ב-145 אלף שקלים לגרסת הסדאן, משלבת היבואנית גרסת כניסה מוזלת, גם אם עדיין יקרה ביחס לשוק. דלק מוטורס הודיעה היום (ד\') כי גרסת הבסיס לדגם תהיה במרכב סדאן ברמת אבזור Comfort ובמחיר של 138 אלף שקלים, זול ב-7,000 שקלים ממחיר הכניסה עד כה. יחידת ההנעה תורכב גם כאן מ-2.0 ליטרים אטמוספרי (165 כ"ס) ותיבת שישה הילוכים אוטומטית.בדרך להוזלה השילה מעצמה ה-3 מספר פריטי אבזור ביחס לגרסת Spirit הסביבית עד כה, בהם מנופי שליטה על תיבת ההילוכים מאחורי ההגה, כניסה ללא מפתח, גג שמש חשמלי, מראה נגד סנוור, פנסים עוקבי פנייה, תאורת לד מאחור ופנסי לד לתאורת היום. היא גם תשווק עם חישוקי "16 קטנים יותר וצמיגים צנועים יחסית במידה 205-60. יש לציין כי בחירה בתמהיל חישוק-צמיג זה עשויה להתגלות כמשפרת נוחות דווקא, לצד פגיעה מסוימת במראה.כמו בגרסאות 3 אחר

KeyboardInterrupt: 